<a href="https://colab.research.google.com/github/vyomakesh0728/telugu_tts/blob/main/telugu_emotion_speech_transcription.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1. Install dependencies
!pip install -q kagglehub openai-whisper polars tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 16.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.5 MB/

In [2]:
# 2. Download dataset
import kagglehub, pathlib, glob, polars as pl, tqdm, os, shutil

ds_path = pathlib.Path(
    kagglehub.dataset_download("jettysowmith/telugu-emotion-speech")
)
wav_files = sorted(glob.glob(str(ds_path / "**/*.wav"), recursive=True))
print(wav_files[:5])

100%|██████████| 201M/201M [00:01<00:00, 119MB/s]

Extracting files...


['/root/.cache/kagglehub/datasets/jettysowmith/telugu-emotion-speech/versions/1/telugu/angry/S11_ALA_C01_G1_D02_SPKM04_V1_AN5_MMM.wav', '/root/.cache/kagglehub/datasets/jettysowmith/telugu-emotion-speech/versions/1/telugu/angry/S11_ALA_C05_G1_D06_SPKF03_V1_AN5_MMM.wav', '/root/.cache/kagglehub/datasets/jettysowmith/telugu-emotion-speech/versions/1/telugu/angry/S11_ALA_C06_G2_D03_SPKF03_V1_AN5_MMM.wav', '/root/.cache/kagglehub/datasets/jettysowmith/telugu-emotion-speech/versions/1/telugu/angry/S11_ALA_C10_G1_D01_SPKF03_V1_AN5_MMM.wav', '/root/.cache/kagglehub/datasets/jettysowmith/telugu-emotion-speech/versions/1/telugu/angry/S11_ALA_C14_G1_D05_SPKM05_V2_AN5_MMM.wav']


In [3]:
# 3. Prepare output dir for reconstructed wavs
out_dir = "wav_bytes_reconstructed"
os.makedirs(out_dir, exist_ok=True)


In [4]:
# 4. Load checkpoint if exists, else start new
checkpoint_path = "telugu_emotion_partial_with_wav.parquet"
rows = []
done = set()
try:
    prev_df = pl.read_parquet(checkpoint_path)
    rows = list(prev_df.iter_rows())
    done = set(prev_df["audio_path"])
    print(f"Resuming from checkpoint: {len(rows)} files already processed.")
except Exception as e:
    print("No checkpoint found. Starting fresh.")


No checkpoint found. Starting fresh.


In [5]:
# 5. Load Whisper (large-v3)
import whisper, torch

device = "cuda" if torch.cuda.is_available() else "cpu"
compute_type = "float16" if device == "cuda" else "int8"

model = whisper.load_model("large-v3", device=device)






100%|█████████████████████████████████████| 2.88G/2.88G [01:05<00:00, 47.1MiB/s]


In [6]:
# 6. Loop over all .wav files; write results after each
for idx, w in enumerate(tqdm.tqdm(wav_files)):
    # Use only files not already processed
    filename = f"audio_{idx+1:05d}.wav"
    new_wav_path = os.path.join(out_dir, filename)
    if new_wav_path in done:
        continue
    try:
        # Read audio bytes and reconstruct into out_dir
        with open(w, "rb") as f:
            wav_bytes = f.read()
        with open(new_wav_path, "wb") as f:
            f.write(wav_bytes)
        # Transcribe with Whisper
        result = model.transcribe(new_wav_path, language="te", fp16=True)
        text = result["text"].strip()
        # Save result immediately to rows and to checkpoint
        rows.append((wav_bytes, text))
        pl.DataFrame(rows, schema=["audio", "text"]).write_parquet(checkpoint_path)
    except Exception as e:
        print(f"Error processing {w}: {e}")
        continue

  0%|          | 0/458 [00:00<?, ?it/s]<ipython-input-6-1726fd7e7b61>:19: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  pl.DataFrame(rows, schema=["audio", "text"]).write_parquet(checkpoint_path)
  0%|          | 2/458 [00:12<43:16,  5.69s/it]  

Error processing /root/.cache/kagglehub/datasets/jettysowmith/telugu-emotion-speech/versions/1/telugu/angry/S11_ALA_C05_G1_D06_SPKF03_V1_AN5_MMM.wav: unexpected value while building Series of type Binary; found value of type String: "ఏయ్ గన్ను నాదగరు ఉంటే కోషలు నువ్ ఏస్తావేంట్ రా ఇరిటేట్ చేకు ఇరిటేట్ చేస్తే చంపేస్తా ఓకే"

Hint: Try setting `strict=False` to allow passing data with mixed types.


<ipython-input-6-1726fd7e7b61>:19: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  pl.DataFrame(rows, schema=["audio", "text"]).write_parquet(checkpoint_path)
  1%|          | 3/458 [00:19<45:03,  5.94s/it]<ipython-input-6-1726fd7e7b61>:19: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  pl.DataFrame(rows, schema=["audio", "text"]).write_parquet(checkpoint_path)
  1%|          | 4/458 [00:21<35:36,  4.71s/it]<ipython-input-6-1726fd7e7b61>:19: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  pl.DataFrame(rows, schema=["audio", "text"]).write_parquet(checkpoint_path)
  1%|          | 5/458 [00:32<49:58,  6.62s/it]<ipython-input-6-1726f

Error processing /root/.cache/kagglehub/datasets/jettysowmith/telugu-emotion-speech/versions/1/telugu/sad/S45_SRI_C01_G2_D04_SPKF21_V1_SA4_MMM.wav: Failed to load audio: ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnap

<ipython-input-6-1726fd7e7b61>:19: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  pl.DataFrame(rows, schema=["audio", "text"]).write_parquet(checkpoint_path)
 73%|███████▎  | 334/458 [1:24:44<33:34, 16.25s/it]<ipython-input-6-1726fd7e7b61>:19: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  pl.DataFrame(rows, schema=["audio", "text"]).write_parquet(checkpoint_path)
 73%|███████▎  | 335/458 [1:25:13<40:34, 19.80s/it]<ipython-input-6-1726fd7e7b61>:19: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  pl.DataFrame(rows, schema=["audio", "text"]).write_parquet(checkpoint_path)
 73%|███████▎  | 336/458 [1:25:49<49:13, 24.21s/it]<ipython-i

In [7]:

# 7. Final save
final_path = "telugu_emotion_speech_allfiles_with_wav.parquet"
pl.DataFrame(rows, schema=["audio", "text"]).write_parquet(final_path)
print(f"Done! Total processed: {len(rows)} files. Output saved to {final_path}")

<ipython-input-7-8963bcd5a5c5>:3: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  pl.DataFrame(rows, schema=["audio", "text"]).write_parquet(final_path)


Done! Total processed: 456 files. Output saved to telugu_emotion_speech_allfiles_with_wav.parquet


In [8]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `edoti` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `edoti`


In [10]:
from datasets import Dataset, Features, Audio, Value
from pathlib import Path
import polars as pl

# -----------------
# 1. Build a Dataset object
# -----------------
df = pl.read_parquet("/content/telugu_emotion_speech_allfiles_with_wav.parquet")        # or build the list of dicts yourself
features = Features({"audio": Audio(), "text": Value("string")})

hf_ds = Dataset.from_dict(df.to_dict(as_series=False), features=features)

# -----------------
# 2. Push to the Hub
#    – option A: keep everything inside Parquet
# -----------------
hf_ds.push_to_hub(
    "dvyomkesh/telugu_es_transcription",
    embed_external_files=True,   # default; safe to omit
    private=False                # or True
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/456 [00:00<?, ? examples/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]